In [25]:
from chessdotcom import get_player_game_archives
import chess
import requests

In [23]:
response = get_player_game_archives("fabianocaruana")

# response.json["archives"] gets the urls of a particular players games by month
games = requests.get(url=response.json["archives"][0])

games.json()["games"][0]["pgn"]

https://api.chess.com/pub/player/fabianocaruana/games/2013/03


'[Event "Live Chess"]\n[Site "Chess.com"]\n[Date "2013.03.17"]\n[Round "-"]\n[White "FabianoCaruana"]\n[Black "hastily"]\n[Result "1-0"]\n[CurrentPosition "3r4/p6p/Q2b2p1/3Rk2N/P1p1P3/5P2/1P2K1PP/7R b - -"]\n[Timezone "UTC"]\n[ECO "D10"]\n[ECOUrl "https://www.chess.com/openings/Slav-Defense-3.Nc3-dxc4"]\n[UTCDate "2013.03.17"]\n[UTCTime "15:16:14"]\n[WhiteElo "1363"]\n[BlackElo "1193"]\n[TimeControl "180"]\n[Termination "FabianoCaruana won by resignation"]\n[StartTime "15:16:14"]\n[EndDate "2013.03.17"]\n[EndTime "15:19:21"]\n[Link "https://www.chess.com/game/live/474897192"]\n\n1. d4 {[%clk 0:03:00]} 1... d5 {[%clk 0:03:00]} 2. c4 {[%clk 0:02:57.8]} 2... dxc4 {[%clk 0:02:57.2]} 3. Nc3 {[%clk 0:02:56.9]} 3... c6 {[%clk 0:02:56.5]} 4. a4 {[%clk 0:02:54.8]} 4... Nf6 {[%clk 0:02:52.7]} 5. e4 {[%clk 0:02:53.5]} 5... c5 {[%clk 0:02:48.1]} 6. d5 {[%clk 0:02:51.5]} 6... Bg4 {[%clk 0:02:44]} 7. f3 {[%clk 0:02:49.7]} 7... Bh5 {[%clk 0:02:41.6]} 8. Bxc4 {[%clk 0:02:48.7]} 8... Qa5 {[%clk 0:02:37

Way to get games:
1) Find any 1500ish player
2) Get a random game
3) Go into their opponents account and get a random game
4) Repeat
5) Compile all of these games to make a set

Things to consider:
- Ratings are wrong when accounts are new or have low game numbers
- Pick games from unique players
- Python chess has opening and endgame databases
https://python-chess.readthedocs.io/en/latest/

Action Items:
- Parse PGN
- Find an efficient way to gather data
    - Make the dataset of PGNs
- Compute time differences between moves
- Make a "stage of game" variable to classify opening/mid/endgame
- EDA of times. Summary statistics of time vs stage of game, etc
